# Model 1: BERTopic to with bill text to predict policy area

In [1]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers[tf-cpu] --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 27.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 7.5 MB/s eta 0:00:00
 

In [2]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 23.4 MB/s eta 0:00:00


In [3]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [4]:
import numpy as np
import tensorflow as tf
#from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
#import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()
#from transformers import TFBertModel
import sklearn as sk

## Load data

In [5]:
# Load the Drive helper and mount
#from google.colab import drive

# This will prompt for authorization.
#drive.mount('/content/drive')

In [6]:
#!ls "/content/drive/My Drive/MIDS-266/final_project"

In [5]:
import pandas as pd

#train = pd.read_csv('/content/drive/My Drive/MIDS-266/final_project/train.csv')
#test = pd.read_csv('/content/drive/My Drive/MIDS-266/final_project/test.csv')

from datasets import load_dataset
train_dataset = load_dataset("AmazonScience/massive", "en-US", split='train')
test_dataset = load_dataset("AmazonScience/massive", "en-US", split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/11514 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2033 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2974 [00:00<?, ? examples/s]

In [6]:
train = pd.DataFrame.from_dict(train_dataset)
test = pd.DataFrame.from_dict(test_dataset)

In [7]:
# change labels from text to int

labels = set(train["intent"])

def create_label_maps(labels):
    num_to_lab = {}
    lab_to_num = {}
    for i, label in enumerate(labels):
        num_to_lab[i] = label
        lab_to_num[label] = i
    return num_to_lab, lab_to_num

In [8]:
num_to_lab, lab_to_num = create_label_maps(labels)

In [9]:
def create_label_num_lst(label_lst):
    num_lst = []
    for label in label_lst:
        num_lst.append(lab_to_num[label])
    return num_lst

## Build model

In [12]:
#x = list(train["title"])
#labels_lst = list(train["policy_area"])
#y = list(test["title"])
#test_labels = list(test["policy_area"])

In [10]:
x = list(train["utt"])
labels_lst = list(train["intent"])
y = list(test["utt"])
test_labels = list(test["intent"])

In [11]:
labels_lst = create_label_num_lst(labels_lst)
test_labels = create_label_num_lst(test_labels)

In [12]:
from tqdm import tqdm

def to_str(lst):
    l = []
    for v in tqdm(lst):
        try:
            l.append(str(v))
        except:
            continue
    return l

In [13]:
x = to_str(x)#[:5000]
labels_lst = labels_lst#[:5000]
y = to_str(y)#[:5000]
test_labels = test_labels#[:5000]

100%|██████████| 2974/2974 [00:00<00:00, 2468604.81it/s]


In [14]:
num_classes = len(set(labels_lst))
labels_train = tf.keras.utils.to_categorical(labels_lst)
labels_test = tf.keras.utils.to_categorical(test_labels)

In [15]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
bert_model = TFBertModel.from_pretrained('bert-base-cased')

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [16]:
MAX_SEQUENCE_LENGTH = 200

In [17]:
# Tokenize train
bert_train_tokenized = bert_tokenizer(x,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_train_inputs = [bert_train_tokenized.input_ids,
                     bert_train_tokenized.token_type_ids,
                     bert_train_tokenized.attention_mask]
bert_train_labels = np.array(labels_train)

# Test/Validation
bert_test_tokenized = bert_tokenizer(y,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')
bert_test_inputs = [bert_test_tokenized.input_ids,
                     bert_test_tokenized.token_type_ids,
                     bert_test_tokenized.attention_mask]
bert_test_labels = np.array(labels_test)

In [54]:
#len(bert_test_labels[0])

60

In [59]:
from tensorflow.python.keras.layers import Embedding, Input, Dense, Lambda

def create_bert_avg_model(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          hidden_size = 201,
                          dropout=0.3,
                          learning_rate=0.00005,
                          num_classes = 60):
    """
    Build a simple classification model with BERT. Use the average of the BERT output tokens
    """

    bert_model = TFBertModel.from_pretrained('bert-base-cased')

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(**bert_inputs)

    # New code
    pooled_output = bert_out[1]

    # Hidden layer
    hidden_layer = tf.keras.layers.Dense(hidden_size, activation='relu')(pooled_output)
    hidden_layer = tf.keras.layers.Dropout(dropout)(hidden_layer)

    # Classification layer
    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(hidden_layer)

    # Model
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=output_layer)

    # Compile the model
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'],
                  #run_eagerly=True
                  )

    ### END YOUR CODE

    return classification_model

In [61]:
bert_avg_model = create_bert_avg_model()
bert_avg_model.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids_layer (InputLaye  [(None, 200)]                0         []                            
 r)                                                                                               
                                                                                                  
 attention_mask_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                    

In [62]:
bert_avg_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data=(bert_test_inputs, bert_test_labels),
    batch_size=10,
    epochs=1)

500/500 [==============================] - 1515s 3s/step - loss: 1.6619 - accuracy: 0.5906 - val_loss: 3.6678 - val_accuracy: 0.4469


In [63]:
bert_avg_2_model = create_bert_avg_model(learning_rate=0.0001)
bert_avg_2_model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids_layer (InputLaye  [(None, 200)]                0         []                            
 r)                                                                                               
                                                                                                  
 attention_mask_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                    

In [64]:
bert_avg_2_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data=(bert_test_inputs, bert_test_labels),
    batch_size=15,
    epochs=2)

Epoch 1/2
334/334 [==============================] - 1206s 3s/step - loss: 1.5037 - accuracy: 0.6572 - val_loss: 3.6719 - val_accuracy: 0.4506
Epoch 2/2
334/334 [==============================] - 1165s 3s/step - loss: 0.5706 - accuracy: 0.8688 - val_loss: 3.4729 - val_accuracy: 0.4778


1 run w default learning rate, batch_size = 10 and epochs =1,

# BERT using average output

In [76]:
def create_bert_avg_out_model(max_sequence_length=MAX_SEQUENCE_LENGTH,
                              hidden_size = 201,
                              dropout=0.3,
                              learning_rate=0.00005,
                              num_classes = 60):
    """
    Build a simple classification model with BERT. Use the average of the BERT output tokens
    """

    bert_model = TFBertModel.from_pretrained('bert-base-cased')

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(**bert_inputs)

    # New code
    output_tensor = bert_out.last_hidden_state  # Get the output tensor (last hidden state)

    # Average the output vectors for all tokens in the full sequence
    avg_output = tf.math.reduce_mean(output_tensor, axis=1)

    # Hidden layer
    hidden_layer = tf.keras.layers.Dense(hidden_size, activation='relu')(avg_output)
    hidden_layer = tf.keras.layers.Dropout(dropout)(hidden_layer)

    # Classification layer
    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(hidden_layer)

    # Model
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=output_layer)

    # Compile the model
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'],
                  #run_eagerly=True
                  )

    ### END YOUR CODE

    return classification_model

In [77]:
bert_avg_3_model = create_bert_avg_out_model()
bert_avg_3_model.summary()

Model: "model_13"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids_layer (InputLaye  [(None, 200)]                0         []                            
 r)                                                                                               
                                                                                                  
 attention_mask_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                   

In [78]:
bert_avg_3_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data=(bert_test_inputs, bert_test_labels),
    batch_size=10,
    epochs=1)

500/500 [==============================] - 1486s 3s/step - loss: 1.0586 - accuracy: 0.7440 - val_loss: 3.4003 - val_accuracy: 0.4839


In [91]:
bert_avg_4_model = create_bert_avg_out_model(learning_rate=0.0001)
bert_avg_4_model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids_layer (InputLaye  [(None, 200)]                0         []                            
 r)                                                                                               
                                                                                                  
 attention_mask_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                   

In [92]:
bert_avg_4_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data=(bert_test_inputs, bert_test_labels),
    batch_size=10,
    epochs=3)

Epoch 1/3


500/500 [==============================] - 1513s 3s/step - loss: 1.1233 - accuracy: 0.7266 - val_loss: 3.6100 - val_accuracy: 0.4654
Epoch 2/3
500/500 [==============================] - 1471s 3s/step - loss: 0.4821 - accuracy: 0.8820 - val_loss: 3.7951 - val_accuracy: 0.4755
Epoch 3/3
500/500 [==============================] - 1469s 3s/step - loss: 0.3624 - accuracy: 0.9114 - val_loss: 4.7088 - val_accuracy: 0.4667


# BERT + CNN

In [18]:
def create_bert_cnn_model(max_sequence_length=MAX_SEQUENCE_LENGTH,
                          num_filters = [131, 127, 51, 23, 17],
                          kernel_sizes = [2, 3, 4, 5, 7],
                          dropout = 0.3,
                          hidden_size = 201, #275, #100
                          learning_rate=0.00005,
                          num_classes=60):
    """
    Build a  classification model with BERT, where you apply CNN layers  to the BERT output
    """
    bert_model = TFBertModel.from_pretrained('bert-base-cased')

    ### YOUR CODE HERE
    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    # BERT Model
    bert_out = bert_model(bert_inputs)
    output_tensor = bert_out.last_hidden_state

    # CNN Layers
    cnn_outputs = []
    for num_filter, kernel_size in zip(num_filters, kernel_sizes):
        conv_layer = tf.keras.layers.Conv1D(filters=num_filter, kernel_size=kernel_size, activation='relu')(output_tensor)
        max_pool_layer = tf.keras.layers.GlobalMaxPooling1D()(conv_layer)
        cnn_outputs.append(max_pool_layer)

    concatenated_cnn_outputs = tf.keras.layers.Concatenate()(cnn_outputs)

    # Fully Connected Layers
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(concatenated_cnn_outputs)
    hidden = tf.keras.layers.Dropout(dropout)(hidden)

    output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(hidden)

    # Model
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[output_layer])

    # Compilation
    # Compile the model
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'],
                  run_eagerly=True
                  )

    ### END YOUR CODE

    return classification_model

In [74]:
bert_cnn_model = create_bert_cnn_model()
bert_cnn_model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 200)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                   

In [75]:
bert_cnn_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data=(bert_test_inputs, bert_test_labels),
    batch_size=10,
    epochs=1)

  1/500 [..............................] - ETA: 48:00 - loss: 5.1237 - accuracy: 0.0000e+00

  2/500 [..............................] - ETA: 39:40 - loss: 4.9818 - accuracy: 0.0000e+00

  3/500 [..............................] - ETA: 39:19 - loss: 4.9159 - accuracy: 0.0000e+00

  4/500 [..............................] - ETA: 39:07 - loss: 4.8686 - accuracy: 0.0250    

  5/500 [..............................] - ETA: 39:03 - loss: 4.6236 - accuracy: 0.0200

...

496/500 [============================>.] - ETA: 19s - loss: 1.2612 - accuracy: 0.6948

497/500 [============================>.] - ETA: 14s - loss: 1.2640 - accuracy: 0.6946

498/500 [============================>.] - ETA: 9s - loss: 1.2643 - accuracy: 0.6948 

499/500 [============================>.] - ETA: 4s - loss: 1.2638 - accuracy: 0.6948

500/500 [==============================] - 2841s 6s/step - loss: 1.2624 - accuracy: 0.6950 - val_loss: 3.2038 - val_accuracy: 0.4785


In [19]:
bert_cnn_2_model = create_bert_cnn_model(learning_rate=0.0001)
bert_cnn_2_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 200)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 200)]                0         []                            
 tLayer)                                                                                      

In [20]:
bert_cnn_2_model.fit(
    bert_train_inputs,
    bert_train_labels,
    validation_data=(bert_test_inputs, bert_test_labels),
    batch_size=10,
    epochs=2)

Output hidden; open in https://colab.research.google.com to view.